In [1]:
import cPickle as pickle
import pandas as pd
from collections import defaultdict, Counter
import numpy as np
from tqdm import tqdm

In [2]:
data=pickle.load(open("results_.pickle", 'rb'))

In [3]:
data.head(5)

,corpus_id,docid,ngram_id,count,doc_length,tfidf,tf,tfidf_var,tfidf_fullvar,tfidf_HHI
0,0,137827,23737,107,11872,0.050491,0.009013,0.039271,1.172770e-08,0.050491
1,0,137827,62,89,11872,0.014588,0.007497,0.012235,7.028444e-07,0.000000
2,0,137827,488,67,11872,0.016907,0.005644,0.014718,9.472815e-10,0.000000
3,0,137827,22152,53,11872,0.021899,0.004464,0.017032,2.374237e-11,0.021899
4,0,137827,137671,39,11872,0.018403,0.003285,0.014314,5.678782e-10,0.018403


In [5]:
docid_ngram_dict=defaultdict(list)
for docid in data['docid'].drop_duplicates().tolist():
    docid_ngram_dict[docid]=data[data['docid']==docid]['ngram_id'].drop_duplicates().tolist()

In [6]:
corpus_ngram_dict=defaultdict(list)
for corpus in range(3):
    corpus_ngram_dict[corpus]=data[data['corpus_id']==corpus]['ngram_id'].drop_duplicates().tolist()

In [7]:
methods=data.columns[5:]
method_list=methods.tolist()

In [8]:
method_list

['tfidf', 'tf', 'tfidf_var', 'tfidf_fullvar', 'tfidf_HHI']

In [9]:
corpus_list=range(3)

In [10]:
ngram_corpus_dict=defaultdict(list)
for corpus in corpus_list:
    ngram_list=corpus_ngram_dict[corpus]
    for ngram in tqdm(ngram_list):
        ngram_corpus_dict[ngram].append(corpus)

100%|██████████████████████████████████████████████| 8689/8689 [00:00<?, ?it/s]


In [11]:
Counter(map(lambda x: len(ngram_corpus_dict[x]), data['ngram_id'].drop_duplicates().tolist()))

Counter({1: 152231, 2: 5582, 3: 389})

In [12]:
ngram_in_1_corpus=filter(lambda x: len(ngram_corpus_dict[x])==1, data['ngram_id'].drop_duplicates().tolist())

In [13]:
docid_corpus_fact=defaultdict()
df=pd.concat([data['docid'], data['corpus_id']],axis=1).drop_duplicates()
for _, row in df.iterrows():
    docid_corpus_fact[row['docid']]=[0,0,0]
    docid_corpus_fact[row['docid']][row['corpus_id']]=1

In [14]:
docid_corpus_fact

defaultdict(None,
            {217: [1, 0, 0],
             693: [1, 0, 0],
             791: [1, 0, 0],
             1272: [1, 0, 0],
             1322: [0, 1, 0],
             1336: [1, 0, 0],
             1441: [0, 1, 0],
             1464: [0, 1, 0],
             1472: [0, 0, 1],
             2100: [1, 0, 0],
             2246: [0, 0, 1],
             2419: [1, 0, 0],
             2618: [0, 1, 0],
             2689: [1, 0, 0],
             2971: [0, 1, 0],
             3275: [0, 0, 1],
             3336: [1, 0, 0],
             3523: [0, 0, 1],
             4559: [0, 0, 1],
             4575: [0, 1, 0],
             4790: [1, 0, 0],
             4983: [0, 1, 0],
             5031: [0, 0, 1],
             5310: [1, 0, 0],
             5372: [1, 0, 0],
             5449: [0, 1, 0],
             5777: [1, 0, 0],
             6022: [1, 0, 0],
             6193: [0, 1, 0],
             6228: [1, 0, 0],
             6378: [0, 0, 1],
             6417: [0, 0, 1],
             6472: [0, 1,

In [19]:
ngram_corpus_results=defaultdict()
for method in method_list:
    ngram_corpus_results[method]=defaultdict(list)

for ngram in tqdm(ngram_corpus_dict.iterkeys()):
    for method in method_list:
        count=[0,0,0]
        for corpus in ngram_corpus_dict[ngram]:
            tf_list=data[(data['ngram_id']==ngram)&(data['corpus_id']==corpus)][method].tolist()
            count[corpus]=np.mean(tf_list)
        ngram_corpus_results[method][ngram]=count
        
        
    

158202it [23:15, 113.36it/s]


In [20]:
document_corpus_results_one_value=defaultdict()

document_corpus_results_all_values=defaultdict()
    
for docid in tqdm(docid_ngram_dict.iterkeys()):
    
    document_corpus_results_one_value[docid]=defaultdict(list)
    document_corpus_results_all_values[docid]=defaultdict(list)
    
    for method in method_list:
        count_one_value=[0,0,0]
        count_all_values=[0,0,0]
        
        for ngram in docid_ngram_dict[docid]:
        
            #most meaningfull one
            max_value=max(ngram_corpus_results[method][ngram])
            corpus_index=ngram_corpus_results[method][ngram].index(max_value)
            count[corpus_index]+=data[(data['ngram_id']==ngram)&(data['docid']==docid)]['count'].tolist()[0]
            
            document_corpus_results_one_value[docid][method]=count_one_value
            
            #every word
            for index, value in enumerate(ngram_corpus_results[method][ngram]):
                count_all_values[index]=value*(data[(data['ngram_id']==ngram)&(data['docid']==docid)]['count'].tolist()[0])
                
            document_corpus_results_all_values[docid][method]=count_all_values
                
            

537it [2:32:32, 24.27s/it]


In [21]:
pickle.dump(ngram_corpus_results, open('ngram_results.pickle', 'wb'))
pickle.dump(document_corpus_results_one_value, open('document_results_1v.pickle', 'wb'))

In [22]:
pickle.dump(document_corpus_results_all_values, open('document_results_allv.pickle', 'wb'))